In [1]:
from os.path import join, abspath, expanduser
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import datetime
from rio_geom import rio_to_exterior
from datetime import timedelta
import geopandas as gpd
import rioxarray as rxa
import contextily as ctx

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data_dir = expanduser('~/scratch/data/uavsar/snowpits')
swe = pd.read_csv(join(data_dir, 'SNEX20_TS_SP_Summary_SWE_v01.csv'), parse_dates = ['Date/Local Standard Time'], index_col= 'PitID')

env = pd.read_csv(join(data_dir, 'SNEX20_TS_SP_Summary_Environment_v01.csv'), parse_dates = ['Date/Local Standard Time'], index_col= 'PitID')

pit_sum = pd.concat([swe, env], axis = 1).reindex(swe.index)
pit_sum = pit_sum.loc[:,~pit_sum.columns.duplicated()]
pit_sum = pit_sum.rename(columns = {'Date/Local Standard Time':'date'})
pit_sum = gpd.GeoDataFrame(pit_sum, geometry=gpd.points_from_xy(pit_sum['Longitude (deg)'], pit_sum['Latitude (deg)']), crs = 4326)

In [3]:
with open(expanduser('~/scratch/data/uavsar/image_fps'), 'rb') as f:
    image_fps = pickle.load(f)

res = pd.DataFrame()
for i, image_fp in enumerate(image_fps):
    if i == i:
        dic = {}
        dic['image_data'] = image_fp
        loc = image_fp['location']
        dic['loc'] = loc
        if image_fp['flight1'].date().year == 2021 or image_fp['pol'] != 'HH':
            pass
        else:
            geom = rio_to_exterior(image_fp['cor'])
            for i, dt in enumerate(['flight1', 'flight2']):
                dic[f't{i+1}'] = image_fp[dt]
                dt = image_fp[dt].date()
                # Form a date range to query on either side of our chosen day 
                date_range = [dt + i * timedelta(days=1) for i in [-2, 0, 2]]
                df_range = pit_sum[(pit_sum.date.dt.date > date_range[0]) & (pit_sum.date.dt.date < date_range[-1])]

                if len(df_range) > 0:
                    # View snow pits that are +/- 1 day of the first UAVSAR flight date
                    geom = geom.to_crs(df_range.crs)
                    points_within = gpd.sjoin(df_range, geom, predicate='within')
                    points_within = points_within.drop(['index_right'], axis=1)
                    dic[f't{i+1}_pit_num'] = len(points_within)
                    dic[f't{i+1}_pits'] = points_within
                else:
                    dic[f't{i+1}_pit_num'] = len(points_within)
                    print(f'{dt}_{loc} has none')
            res = pd.concat([res, pd.DataFrame.from_dict([dic])])

In [ ]:
                    # for label in ['fp','inc','cor','hgt']:
                    #     img = rxa.open_rasterio(img_set[label])
                    #     site_name = site['name']
                    #     if label == 'fp':
                    #         label = 'unw'
                    #     dic[label] = img.sel(x = site.geometry.x, y = site.geometry.y, method = 'nearest', tolerance = 0.0001).values[0]


In [29]:
for i,r in res.iterrows():
    if not r.t1_pits.empty and not r.t2_pits.empty:
        epsg = 26900 + int(r.t1_pits.iloc[0]['UTM Zone'].replace('N',''))
        merged = gpd.sjoin_nearest(r.t1_pits.to_crs(epsg), r.t2_pits.to_crs(epsg), max_distance = 50, lsuffix='t1', rsuffix='t2')
        merged['sd_diff'] = merged['Snow Depth (cm)_t2'] - merged['Snow Depth (cm)_t1']
        merged['swe_diff'] = merged['SWE (mm)_t2'] - merged['SWE (mm)_t1']
for label in ['fp','inc','cor','hgt']:
                    #     img = rxa.open_rasterio(img_set[label])
                    #     site_name = site['name']
                    #     if label == 'fp':
                    #         label = 'unw'
                    #     dic[label] = img.sel(x = site.geometry.x, y = site.geometry.y, method = 'nearest', tolerance = 0.0001).values[0]

        # print(f'{r.t1_pit_num} - {r.t2_pit_num}')
        # print(len(merged))
        # merged = pd.concat([r['t1_pits'].set_index('Site').add_suffix('_t1'), r['t2_pits'].set_index('Site').add_suffix('_t2')], axis = 1).dropna(subset = ['Location_t1','Location_t2'])